In [1]:
import sys
print({sys.executable})
!{sys.executable} -m pip list | grep -E "torch|cuda|datasets|pandas|tqdm|numpy|pyarrow|transformers|tokenizers|trl|unsloth|unsloth_zoo|huggingface-hub|datasets|numpy|triton"

{'/home/zeus/miniconda3/envs/cloudspace/bin/python'}
datasets                  4.3.0
huggingface-hub           0.34.4
numpy                     2.3.4
nvidia-cuda-cupti-cu12    12.8.90
nvidia-cuda-nvrtc-cu12    12.8.93
nvidia-cuda-runtime-cu12  12.8.90
pandas                    2.3.3
pyarrow                   22.0.0
pytorch-lightning         2.6.0
tokenizers                0.22.1
torch                     2.8.0+cu128
torchao                   0.15.0
torchmetrics              1.7.4
torchvision               0.23.0+cu128
tqdm                      4.67.1
transformers              4.56.2
triton                    3.4.0
trl                       0.22.2
unsloth                   2025.12.9
unsloth_zoo               2025.12.7


In [2]:
!{sys.executable} -m pip list  | grep triton

triton                    3.4.0


In [3]:
#!uv pip install --upgrade --no-deps bitsandbytes transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo huggingface-hub==0.34.4 datasets==4.3.0 tqdm==4.67.1 

In [4]:
#!pip install --upgrade  uv

In [5]:
#!uv pip install "triton>=3.4.0" bitsandbytes "transformers==4.56.2" \
#"unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
#"unsloth[base] @ git+https://github.com/unslothai/unsloth" \
#        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels

In [6]:
import torch
print("Pre-warmed imports")

Pre-warmed imports


In [7]:
import sys
!{sys.executable} -m pip list | grep torch
!{sys.executable} -m uv pip list | grep torch

pytorch-lightning         2.6.0
torch                     2.8.0+cu128
torchao                   0.15.0
torchmetrics              1.7.4
torchvision               0.23.0+cu128
Using Python 3.12.11 environment at: /home/zeus/miniconda3/envs/cloudspace
pytorch-lightning         2.6.0
torch                     2.8.0+cu128
torchao                   0.15.0
torchmetrics              1.7.4
torchvision               0.23.0+cu128


In [8]:
import sys, json
from pathlib import Path

print("Current Python executable:")
print(sys.executable)

kernel_name = Path(sys.prefix).name
print("\nActive env name:", kernel_name)

print("\nKernel spec info:")
!jupyter kernelspec list


Current Python executable:
/home/zeus/miniconda3/envs/cloudspace/bin/python

Active env name: cloudspace

Kernel spec info:
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Available kernels:
  python3    /home/zeus/miniconda3/envs/cloudspace/share/jupyter/kernels/python3


In [9]:
!uv pip list 

Using Python 3.12.11 environment at: /home/zeus/miniconda3/envs/cloudspace
Package                   Version
------------------------- ------------
absl-py                   2.3.1
accelerate                1.12.0
aiohappyeyeballs          2.6.1
aiohttp                   3.13.2
aiosignal                 1.4.0
annotated-doc             0.0.4
annotated-types           0.7.0
anyio                     4.12.0
argon2-cffi               25.1.0
argon2-cffi-bindings      25.1.0
arrow                     1.4.0
asttokens                 3.0.1
async-lru                 2.0.5
attrs                     25.4.0
babel                     2.17.0
backoff                   2.2.1
beautifulsoup4            4.14.3
bitsandbytes              0.49.0
bleach                    6.3.0
boto3                     1.42.3
botocore                  1.42.3
cachetools                6.2.2
certifi                   2025.11.12
cffi                      2.0.0
charset-normalizer        3.4.4
click                     8.3.1
comm

In [10]:
#!uv pip install --upgrade --no-deps \
#    transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

# Before everything we start,  preapre the envionment 

## Best Practice to follow : 

- Use Python 3.10 in your runtime
- Install all heavy binary libs from conda
- Install pip-only pure python packages AFTER NumPy/Torch are settled
- Install optional Jupyter kernel inside the environment
- (Optional but recommended) Create a reproducible environment file
- Use Docker instead If your server supports Docker + NVIDIA Container Toolkit, this is the most reproducible

✅ NEVER mix uv/pip with conda for GPU ML frameworks (PyTorch, CUDA, Triton, bnb, xformers). 
You will see such kind of installation on jupyter notebook tutorial, that's a different story.

✅ !uv pip install only gets the installation faster due to the uv.  It deson't support --dry-run to check if there are any related dependency packages will be updated.

✅ How to lock/freeze the enviornment 

environment.yml (micromamba/conda) is the most solid choice

| Method                  | System Packages | Python Packages |  Sub-dependencies | Reproducible? |
|-------------------------|-------------------------|-------------------------|---------------------------|---------------|
| pip freeze              | ❌                       | ✅                       | ❌                         | Partial       |
| pip-tools / Poetry       | ❌                       | ✅                       | ✅                         | ✅            |
| environment.yml  | ✅           | ✅                       | ✅                         | ✅            |


✅ In Jupyter notebook,  pip/uv pip install actually not install packages into the connected kernel python environment. 
!pip list show different results from terminal (after  activate the env of the kernel) ??

if you used !pip inside the notebook, it sometimes runs the system pip, not the pip from the kernel environment. 

!pip is a shell command. It resolves pip from the PATH in the Jupyter server process.

That PATH might point to your system Python (or Jupyter’s default env) not your kernel’s env.

That PATH might point to your system Python (or Jupyter’s default env) not your kernel’s env.

!pip in notebooks is not fully reliable; always use {sys.executable} -m pip to be sure.

```python
import sys
!{sys.executable} -m pip list
!{sys.executable} -m pip install package-name
!{sys.executable} -m uv pip list
!{sys.executable} -m uv pip install some-package
```

✅ You will see a lot `!pip install !uv pip install` in the tutorial in jupyter notebook, if you are using the default system kernel, it's fine.  But not the customized envionment


## For strick production ML enviornment

### Packages contain large compiled C/CUDA binaries, should comes from Conda/micomamba 

> ABI compatibility  Application Binary Interface, how compiled components talk to each other.
> PyTorch links to NumPy C-API
> pandas links to NumPy C-API
> PyArrow links to libarrow + NumPy C-API
> PyTorch + TorchVision + TorchAudio + CUDA must be built with same compiler, same runtime, same dependencies
> *If any one of these comes from pip, you may get a wheel compiled against a different compiler, different NumPy ABI,
> different glibc, etc → binary mismatch errors*

### Conda/micomamba solves dependency versions together

conda/mamba/micomamba solves installation as a set so they are fully compatible.

pip cannot solve binary dependencies; it installs whatever wheels exist.

### Conda binaries are built with compatible compilers

conda-forge and pytorch channels use:

same clang/gcc

same GLIBC

same CUDA runtime ABI

same Arrow C++ libs

same NumPy C API version

pip wheels might:

be compiled with gcc 9, 10, or 11

assume NumPy 1.x ABI instead of 2.x

bundle their own incompatible libarrow

So mixing pip wheels for heavy packages is dangerous.

### packages 

Which packages must ALWAYS come from conda?

(🔥 Critical — do NOT install via pip/uv)

✅ Heavy Compiled Packages
Package	Why it must be from conda
pytorch	CUDA + C++ + MKL linked libs
torchvision, torchaudio	Must match PyTorch ABI
pytorch-cuda	CUDA runtime libs; must match PyTorch
numpy	Core C-API; other libs depend on it
pandas	Compiled against NumPy C-API
pyarrow	Compiles against Arrow + NumPy C-API
scipy (if any)	Huge C++ build, must match NumPy
opencv (cv2)	Must match system libs / NumPy ABI
tqdm (optional)	OK in pip but stable in conda

Which packages can come from pip/uv (safe)

Pure Python packages (no compiled code):

transformers
trl
huggingface-hub
datasets
unsloth
unsloth_zoo
tqdm
tokenizers ❗ ← C++ but HuggingFace wheels are safe with both pip/conda

General rule:

✅ If it does NOT contain heavy C++/CUDA → pip is OK
❌ If it DOES contain C++/CUDA → use conda


## commands 

```cmd

"${SHELL}" <(curl -L micro.mamba.pm/install.sh)

./bin/micromamba shell init -s bash -r ~/micromamba

# better yet, restart your terminal!
source ~/.bashrc

micromamba create -n unsloth -c conda-forge -c nvidia -c pytorch python=3.10.12 pytorch=2.8.0 torchvision torchaudio pytorch-cuda numpy pillow pyarrow pandas  scipy scikit-learn triton==3.4.0

This locks the channel sources for the environment. afterward will automatically use  conda-forge → nvidia  → pytorch  in that order.

micromamba activate unsloth
  
pip install --dry-run --no-deps transformers==4.56.2 tokenizers torchmetrics trl==0.22.2 unsloth unsloth_zoo huggingface-hub==0.34.4 datasets==4.3.0 tqdm==4.67.1

micromamba env export > environment.yml

pip freeze > requirements.txt

python -m ipykernel install --name ml --display-name "ML (python 3.10)"

```
> If you are working on a standard server,  your customized ML stack is done.
> BUT in a cloud VM environment,  things will be much more complicated.  As the storage has a lot of underlying tricks.
> - Some of the storage locations are overlay filesystem of container image.  A Writable layer on top a read only layer.
>   what your saw is combination results of these. Any changes will be lost after next reboot. It's pretty fair with cloud
>   environment, reboot/restart bring a fresh new workable bench.
>   
> - Some of the persistent storage are on a networked overlay filesystem, files are stored on persistent network storage,
>   not a local SSD. Take lighting studio as an example:
>
>   a. Every I/O operation (e.g., imports, reads, writes, temporary cache creation) is slower — sometimes by seconds per call.
>
>   b. When the Studio wakes from “sleep,” it reloads your environment. Your micromamba env may live on a slower layer (overlay),
>   and python and libraries are reloaded into memory the first time you import them — causing the “long first cell execution.”
>
>   Fix: pre-warm import torch, transformers 
>
>   c. IPC lag. In Lightning AI, your Jupyter UI and kernel may sometimes run in separate containers — especially after a restart
>   or upgrade. If your kernel’s python path is /teamspace/studios/this_studio/micromamba/envs/unsloth/bin/python but the JupyterLab
>   process runs in /opt/jupyter/envs/main, IPC (inter-process communication) may traverse a socket bridge, introducing latency.
>

## Lightning AI extra setting for the customized micomamba env kernel


```cmd
#mkdir -p /teamspace/studios/this_studio/.jupyter_kernels
#~ python -m ipykernel install --name ml --display-name "ML (python 3.10)" --prefix=/teamspace/studios/this_studio/.jupyter_kernels

ln -s /teamspace/studios/this_studio/.jupyter_kernels/share/jupyter/kernels/ml  /opt/jupyter/envs/main/share/jupyter/kernels/ml
sudo systemctl restart jupyterlab.service

```
> JUPYTER_PATH is an environment variable that tells Jupyter to look for kernels (and other data files) in extra directories.
> But that doesn't work as that can't reach into application container. (maybe, not verified)
> Install to /teamspace/studios/this_studio/.local/share/jupyter also does not work, as reload the jupyterlab service will restore
> .local/share/jupyter as well.

> Lightning AI tips: Manually customized studio start/stop operation. Like start a app, or clean storage before sleeping...

```cmd
environment.yml  gpt-oss-20b-fine-tune  main.py  micromamba  requirements.txt
⚡ ~ cat  .lightning_studio/on_start.sh
#!/bin/bash

# This script runs every time your Studio starts, from your home directory.

# Logs from previous runs can be found in ~/.lightning_studio/logs/

# List files under fast_load that need to load quickly on start (e.g. model checkpoints).
#
# ! fast_load
# <your file here>

# Add your startup commands below.
#
# Example: streamlit run my_app.py
# Example: gradio my_app.py
echo "[on_start] starting at $(date)"
echo "[on_start] done at $(date)"
⚡ ~ cat  .lightning_studio/on_stop.sh
#!/bin/bash

# This script runs every time your Studio sleeps, from your home directory.

# Logs from previous runs can be found in ~/.lightning_studio/logs/

# Add your shutdown commands below.
#
# Example: docker down my-container
# Example: sudo service mysql stop

echo "[on_stop] cleaning..."
rm -rf /teamspace/studios/this_studio/.cache
rm -rf /teamspace/studios/this_studio/tmp_checkpoints
echo "[on_stop] done."
```


## Understand the errors

### Why cannot import name 'Inf' from 'numpy' — what changed in NumPy?

NumPy 2.0 removed several legacy aliases — np.Inf / np.NINF / np.Infinity / np.NaN (aliases) — in favor of the lowercase forms (np.inf, np.nan). If code (or older compiled extensions) still expects np.Inf, import-time errors or attribute errors appear.

Actionable: If you see cannot import name 'Inf' it’s from code or a compiled extension expecting the old alias. Fix options:

Downgrade to a NumPy 1.x-compatible version (or use the exact NumPy version the wheel was built for), or

Reinstall the dependent extension compiled against the NumPy you intend to use.

### Why ValueError: numpy.dtype size changed happens (ABI / binary mismatch)

This is not a Python-level logic bug — it’s a binary compatibility problem between NumPy C-API and compiled extensions (Cython, C/C++ extensions such as those inside pyarrow, pandas, tokenizers, Triton kernels). The community docs and many troubleshooting threads explain this is the classic symptom when packages were built against a different NumPy header than currently installed.

Actionable (how to fix):

Reinstall NumPy first (or install it from conda), then reinstall packages with C-extensions so they compile/bind against that NumPy.

Better: create a fresh conda env (Python 3.10/3.11) and install binary packages (numpy/pandas/pyarrow/torch) via conda channels before pip installs.

### Which ML components have trouble on Python 3.12?

**2026-01-05 not so scary anymore I guess. Could build workable ML env with python 3.12** 

Triton: long-standing issues/PRs show Triton did not support 3.12 for a while; many projects depending on Triton (custom GPU kernels) are fragile on 3.12. 
GitHub
+1

PyTorch: historically lagged on official wheel support for 3.12; you must check the official PyTorch wheel index for current support (the PyTorch community threads discuss this). If the runtime doesn’t have appropriate PyTorch wheels for 3.12, installs fail or fall back to incompatible builds. 

Bitsandbytes / tokenizers / other wheels: bitsandbytes requires specific Python/PyTorch/CUDA combinations; while PyPI may list Python 3.9+, there are reported issues and source-build workarounds for 3.12. Tokenizers and some Hugging Face pieces also saw 3.12-related 

Important nuance: Some package maintainers do publish 3.12 wheels over time. But uptake is uneven: not guaranteed for every compiled dependency you depend on. That’s why the conservative recommendation is to use Python 3.10 or 3.11 for ML work today.


## Understand package manager 

### What's the difference between mamba and conda

`mamba` = conda, but much faster.
It uses the same package repositories, same commands, same environment format, but solves dependencies using a much faster backend (libsolv, in C++). It is a drop-in replacement for conda.

1. Speed (the biggest difference)

`conda` uses a Python-based dependency solver → slow, especially for large ML stacks.

`mamba` uses libsolv (C++), the same solver used by Fedora/openSUSE → much faster dependency resolution.

✅ For real ML projects:
Creating large environments with PyTorch, numpy, cudatoolkit, transformers, pyarrow, etc. will take minutes with conda and seconds with mamba.

2. Compatibility

`mamba` and conda use the same package format, the same channels, and the same environment specs.

Any conda command can be replaced with mamba

3. Dependency Solving Behavior

Both use the same rules, but:

`conda` may occasionally get stuck in long solve loops.

`mamba` handles big/complex dependency graphs quickly and reliably due to libsolv.

4. Installation Footprint

`mamba` comes from conda-forge as:

`mamba` (standalone replacement for conda commands)

`micromamba` (even smaller, can bootstrap whole envs without installing conda)

For remote GPU servers or Docker images, people often prefer micromamba to keep containers small.


## micromamba

`micromamba` is better for fresh ML environments, cloud servers, and production

⭐ Reason 1 — Pure, zero-bloat installation

Conda installs hundreds of MB of packages even before you create an environment.
Micromamba installs nothing except itself (a 2–3 MB binary).
Perfect for:
- **Cloud VMs**
- **Docker images**
- Remote GPU servers
- Clean reproducible ML environments

⭐ Reason 2 — Safer + more reproducible

Micromamba:

- **Does not auto-update*8
- **Does not auto-install extra conda garbage*8
- Does not pollute your system
- Works entirely inside a single folder (purely isolated)
- This **makes your ML environment exactly reproducible**, which is critical for:

✅ CUDA correctness
✅ Compiled libs
✅ PyTorch environments

⭐ Reason 3 — Faster than both conda and mamba

Micromamba has:

- Instant startup
- Faster solver for environment creation
- Minimal overhead

| Task                                         | Best choice    | Why                               |
| -------------------------------------------- | -------------- | --------------------------------- |
| Existing conda setup                         | **mamba**      | Drop-in replacement, easy         |
| Fresh minimal system (Ubuntu VM, GPU server) | **micromamba** | Fast, small, reproducible         |
| Docker images                                | **micromamba** | The industry standard             |
| Long-term ML env stability                   | **micromamba** | No surprise updates               |
| Local development with GUI tools             | mamba          | Sometimes easier on Windows/macOS |



### what is conda-forge ?

`conda-forge` is a community-driven collection of conda packages, completely separate from the commercial “Anaconda, Inc.” channels.

Think of it as:

✅ A *community-maintained, open-source package repository*
✅ Much **larger and more up-to-date** than the default Anaconda channel
✅ The **standard source for modern ML, scientific, and data engineering stacks**

It’s basically the PyPI of the conda world—but with compiled (pre-built) binaries.

#### Why it's important

1. Most modern ML packages ship to conda-forge first

Examples:

PyTorch
HuggingFace transformers
NumPy
SciPy
PyArrow
Pandas
JAX
FastAPI
UV, ripgrep, and many system-level tools

Many packages don’t even exist in the “defaults” channel.

2. More up-to-date than Anaconda’s default channel

`conda-forge` maintains fast release cycles.

Example:

PyArrow in conda-forge releases within days.
SciPy, NumPy, JAX usually appear in conda-forge first.

3. Better dependency solving

`conda-forge` tries to keep its entire ecosystem compatible:

All packages pinned to consistent versions
All use modern compilers/libraries
Strong automated build infrastructure
This dramatically reduces “binary mismatch” errors.

4. The preferred channel for mamba

`mamba` itself is published and maintained via conda-forge.

Most mamba users run their entire environment using:

`channels`:
  - conda-forge
  - nvidia
  - pytorch

### what is uv

`uv` is essentially a Python tool/environment manager. Think of it as a modern hybrid between:

- pip (for installing Python packages)
- venv / virtualenv (for creating isolated environments)
- pipx (for installing CLI tools in isolated venvs)

Modern, high-performance package manager and installer for Python, written in Rust. It's designed to be a significantly faster, all-in-one replacement for multiple traditional Python tools, including pip, pip-tools, and virtualenv.

-  Blazing Fast Speed: Due to its implementation in Rust, uv is dramatically faster than pip—benchmarks show it can be 10-100x faster for package installation and dependency resolution, especially when leveraging its built-in global cache.

-  Drop-in pip and pip-tools Replacement: It offers a compatibility layer, meaning you can often swap out commands like pip install with uv pip install and immediately benefit from the speed improvements.

-  *Integrated Environment Management: It eliminates the need for a separate virtualenv or venv tool. uv can create, manage, and synchronize virtual environments directly, which is also significantly faster.*

-  Advanced Dependency Resolution: uv is designed with a more robust dependency resolver to better handle complex dependency conflicts and provide clearer error messages

-  Universal Tooling: It aims to replace the functionality of numerous other tools like pipx (for installing command-line tools) and even offers features for Python version management (similar to pyenv).

-  Reproducibility: It natively supports the creation of lockfiles (uv.lock or a requirements.txt equivalent via uv pip compile), which pin the exact versions of all transitive dependencies to ensure consistent environments across all machines.

> **One thing uv doesn't have is --dry-run in pip**
> Which actually pretty useful

#### Core features of uv

Create a Project/Env
    **uv init <project-name>**
    **Initializes a new project with a virtual environment and a pyproject.toml file.**
Create a Venv
    uv venv
    Creates a virtual environment in the current directory (.venv).
Install Packages
    uv pip install <package-name>
    Use the pip compatible interface for installing packages.
Add a Dependency
    uv add <package-name>
    Adds a package to your pyproject.toml and updates the lockfile/environment.
Compile Dependencies
    uv pip compile requirements.in -o requirements.txt
    uv pip compile pyproject.toml -o uv.lock
    Generates a lockfile (requirements.txt) with pinned versions.
Sync Environment
    uv pip sync requirements.txt 
    Synchronizes the virtual environment with the exact versions in the lockfile.
    Replacement for pip install -r requirements.txt, but faster
Sync Environment
    **uv sync**
    The uv sync command (without specifying a file) assumes you are in a project directory and automatically looks for a generated lockfile (like uv.lock or requirements.txt) to synchronize the environment with the source of truth defined by the pyproject.toml file.
Run a Script/Command
    uv run python script.py
    Executes a command within the project's virtual environment *(related with current dir)*.

## Why Colab scripts reinstall numpy and pillow ? 

Because Google Colab already ships preinstalled versions of NumPy & Pillow that are ABI-incompatible with newer PyTorch wheels.

This causes runtime crashes like:
- ValueError: numpy.dtype size changed
- cannot import name 'Inf' from numpy
- binary incompatibility
- C-extension import errors

Colab's system NumPy/Pillow were compiled with different C compilers / binary formats than the new PyTorch wheels you install.

Therefore:

If you install new PyTorch without matching these libs → Python crashes.

✅ Why NumPy + Pillow breaking your environment matters

These two packages are special:

NumPy 
- The entire scientific stack depends on it
- Has compiled C and Avx2 routines
- ABI changes frequently
- If one pkg links to NumPy 2.0 but you have 1.26 → crash

Pillow

- Image decoding
- Has C libraries (libjpeg, zlib)
- PyTorch vision uses Pillow
- Mismatch breaks dataset loading

✅ Why Unsloth specifically cares

Unsloth uses:

PyTorch
Triton
bitsandbytes
tokenizers
datasets (pyarrow)

This stack is extremely sensitive to:

- Python version
- NumPy ABI
- Pillow ABI
- CUDA runtime

Re-installing NumPy & Pillow ensures everything is built with the same ABI.


In [11]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):    
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo huggingface-hub==0.34.4 datasets==4.3.0 numpy==2.3.4 pandas==2.3.3 pyarrow==22.0.0 tqdm==4.67.1

In [12]:
!pip list | grep -E "datasets|pandas|tqdm|numpy|pyarrow|transformers|tokenizers|trl|unsloth|unsloth_zoo|huggingface-hub|datasets|numpy"

datasets                  4.3.0
huggingface-hub           0.34.4
numpy                     2.3.4
pandas                    2.3.3
pyarrow                   22.0.0
tokenizers                0.22.1
tqdm                      4.67.1
transformers              4.56.2
trl                       0.22.2
unsloth                   2025.12.9
unsloth_zoo               2025.12.7


In [13]:
#!{sys.executable} -m pip install scipy==1.16.3  scikit-learn==1.7.2

In [14]:
from huggingface_hub import notebook_login

In [15]:
notebook_login()

In [18]:
# ================================================
# 2️⃣ data format function for different Datasets
# ================================================


from datasets import load_dataset, concatenate_datasets, Dataset
import pandas as pd
import random
import json

# Set a seed for reproducibility
random.seed(42)

INSPECT_ON = False

# Prints ONE random entry
def inspect_random_entry(ds_name, ds_object, force_disp = False):

    if not force_disp:
        if not INSPECT_ON:
            return 
    
    """Randomly selects and prints one raw sample from the dataset for observation."""
    print(f"\n--- INSPECTION: {ds_name} (ONE RANDOM RAW SAMPLE) ---")
    random_index = random.randint(0, len(ds_object) - 1)
    sample = ds_object[random_index]
    print(f"Sample Index: {random_index} Raw Data:")
    print(json.dumps(sample, indent=2, default=str))
    print("-" * 50)
    
# Prints parts of the formatted datasets
def inspect_formatted_sft(ds_name, ds_object, n = 2, disp_len = 50, force_disp = False):

    if not force_disp:
        if not INSPECT_ON:
            return 
    
    for i, entry in enumerate(ds_object):
        if i > n:
            break
        print(f"--- Entry {i+1} ---")
        print(f"Instruction: {entry['instruction'][:disp_len]}...") # Truncating for readability
        print(f"Response: {entry['response'][:disp_len]}")
        print(f"Source: {entry['source']}")
        print("-" * 50)
        print("\n")


#  EXTRACTION FUNCTIONS. Convert various of datasets to  { "instruction": "xxx", "response": "xxx", "source": "xxx" }
def extract_lmsys_winner(example):
    """
    Identifies the winning conversation (model_a or model_b) and extracts the 
    first instruction/response pair from that conversation list.
    """
    winner_key = example.get('winner')
    
    if winner_key == 'model_a':
        conv_list = example.get('conversation_a')
    elif winner_key == 'model_b':
        conv_list = example.get('conversation_b')
    else: # Handles 'tie', 'tie (a/b)', or missing winner by defaulting to A
        conv_list = example.get('conversation_a')
        
    if not conv_list or len(conv_list) < 2:
        return {"instruction": None, "response": None}

    instruction = conv_list[0]['content'] if conv_list[0]['role'] == 'user' else None
    response = conv_list[1]['content'] if conv_list[1]['role'] == 'assistant' else None
    
    return {"instruction": instruction, "response": response, "source": "lmsys_conv"}


def format_ultrachat(example):
    instruction, response = None, None
    
    user_messages = [m['content'] for m in example['messages'] if m['role'] == 'user']
    model_messages = [m['content'] for m in example['messages'] if m['role'] == 'assistant']
    instruction = user_messages[0] if user_messages else None
    response = model_messages[0] if model_messages else None
    
    return {"instruction": instruction, "response": response, "source": "ultrachat"}


def format_dolly(example):
    """
    Formats a single Dolly 15k entry, integrating 'context' into the 'instruction'
    when available. Assumes the primary keys are 'instruction', 'context', and 'response'.
    """
    
    # Core fields for Dolly (will be None/empty string if not present)
    core_instruction = example.get("instruction", "")
    context = example.get("context", "")
    response = example.get("response", "")
    
    # Initialize the instruction to be returned
    final_instruction = ""

    # --- 1. Build the Instruction ---
    
    # Check if context exists and is not just whitespace
    if isinstance(context, str) and context.strip():
        # Prepend the context to the instruction clearly
        final_instruction = (
            f"Context: {context.strip()}\n\n"
            f"Instruction: {core_instruction.strip()}"
        )
    elif isinstance(core_instruction, str) and core_instruction.strip():
        # If no context, just use the core instruction
        final_instruction = core_instruction.strip()
    # Note: If both are empty, final_instruction remains "" (which will be filtered later).

    # --- 2. Keep the Response and Source ---
    
    if isinstance(response, str):
        final_response = response.strip()
    else:
        final_response = response # Keep None/other type for filtering

    return {
        "instruction": final_instruction, 
        "response": final_response, 
        "source": "dolly"
    }



import re 
# Note: You'd need to ensure 'import re' is at the top of your main script 
# or use pd.Series as you did before, but the pure Python approach is cleaner.

def format_GSM8K(example):
    """
    Formats a single GSM8K entry by extracting the question and cleaning 
    the step-by-step answer (solution) by removing calculator tokens and 
    the final answer marker.
    """
    
    # GSM8K uses 'question' for instruction and 'answer' for response
    instruction = example.get("question")
    response = example.get("answer")

    if isinstance(response, str):
        # --- FINAL FIXES FOR GSM8K using pure Python re module ---
        
        # 1. Remove GSM8K Calculator Tokens (e.g., <<16+20=36>>)
        # The re.sub is much cleaner than the pd.Series chaining.
        response = re.sub(r'<<.*?>>', '', response).strip()

        # 2. Remove the GSM8K Final Answer Marker (e.g., #### 40 or #### 12.5)
        # Using [^\n]+ to catch any numerical/decimal/fractional answer following the marker
        response = re.sub(r'####\s*[^\n]+', '', response).strip()
    
    # The 'instruction' only needs to be the question
    if isinstance(instruction, str):
        instruction = instruction.strip()
    
    return {"instruction": instruction, "response": response, "source": "GSM8K"}

def format_OpenThoughts(example):
    instruction, response = None, None

    # Check for the 'conversations' key typical of OpenThoughts structure
    if 'conversations' in example and isinstance(example['conversations'], list):
        # The user's instruction is the 'value' of the first 'user' turn
        # 1. Extract Instruction (User's turn)
        user_turn = next((c for c in example['conversations'] if c['from'] == 'user'), None)
        if user_turn:
            instruction = user_turn.get('value')

        # 2. Extract Response (Assistant's turn)
        assistant_turn = next((c for c in example['conversations'] if c['from'] == 'assistant'), None)
        if assistant_turn:
            response = assistant_turn.get('value')
    

    if isinstance(response, str):
        # Apply the CoT cleaning logic
        response = response.replace("<|begin_of_thought|>", "").replace("<|end_of_thought|>", "").strip()
        response = response.replace("<|begin_of_solution|>", "").replace("<|end_of_solution|>", "").strip()
    
    return {"instruction": instruction, "response": response, "source": "OpenThoughts"}    
    
def format_math(example):
    
    return {"instruction": example["problem"], "response": example["solution"], "source": "math"}


def format_alpaca(example):
    
    # Base instruction
    inst = example.get("instruction", "").strip()
    inp = example.get("input", "")
    out = example.get("output") or example.get("response") or example.get("answer") or ""

    # If input exists and is non-empty, append it with a label
    if isinstance(inp, str) and inp.strip():
        inst = f"{inst}\n\nInput:\n{inp.strip()}"

    # final cleaning
    inst = inst.strip()
    out = out.strip()

    return {"instruction": inst, "response": out, "source": "alpaca"}


def format_sharegpt(example):
    
    if "conversations" not in example or len(example["conversations"]) < 2:
        return {"instruction": None, "response": None}
    convs = example["conversations"]
    user_turns = [c["value"] for c in convs if c["from"] == "human"]
    assistant_turns = [c["value"] for c in convs if c["from"] == "gpt"]
    
    return {
        "instruction": user_turns[0] if user_turns else None,
        "response": assistant_turns[0] if assistant_turns else None,
    }
    
def format_boolq(example):
    
    q = example.get("question", "").strip()
    p = example.get("passage", "").strip()
    a = example.get("answer", None)

    # Convert boolean answer to natural language
    if a is True:
        answer_text = "Yes"
    elif a is False:
        answer_text = "No"
    else:
        answer_text = ""

    # Build structured instruction
    if p:
        instruction = f"Question: {q}\n\nContext:\n{p}"
    else:
        instruction = f"Question: {q}"

    return {"instruction": instruction.strip(), "response": answer_text.strip(), "source": "boolq"}

def format_arc(sample):
    """
    Converts a single sample from the allenai/ai2_arc dataset into 
    the Supervised Fine-Tuning (SFT) instruction-response format.

    Args:
        sample (dict): A single dictionary sample from the dataset.

    Returns:
        dict: A dictionary in the SFT format {"instruction": ..., "response": ...}.
    """
    # 1. Extract the main question and the answer key
    question = sample['question']
    answer_key = sample['answerKey']
    
    # 2. Extract the choices and labels
    choices_text = sample['choices']['text']
    choices_label = sample['choices']['label']
    
    # 3. Create the formatted list of choices (e.g., "A. pulley.\nB. lever.\n...")
    formatted_choices = []
    # Use zip to pair the labels (A, B, C, D) with the corresponding text
    for label, text in zip(choices_label, choices_text):
        formatted_choices.append(f"{label}. {text}")
        
    # Join the choices with newlines to make them easy for the model to read
    choices_block = "\n".join(formatted_choices)
    
    # 4. Construct the full 'instruction' prompt
    instruction = (
        f"Question: {question}\n\n"
        f"Choices:\n{choices_block}\n\n"
        "Please select the letter (A, B, C, or D) that represents the correct answer."
    )
    
    # 5. Determine the correct 'response'
    
    # Find the index of the answer key (e.g., 'B' is at index 1)
    answer_index = choices_label.index(answer_key)
    
    # Get the text corresponding to the answer key
    correct_answer_text = choices_text[answer_index]
    
    # The response should include the key and the full text for a complete answer
    response = f"Correct Answer: {answer_key}. {correct_answer_text}"
    
    # 6. Return the final SFT dictionary
    return {
        "instruction": instruction,
        "response": response,
        "source": "arc"
    }

def format_hotpot(example):
    context_text = "\n".join(example["context"]["sentences"][0]) if "context" in example and isinstance(example["context"], dict) else ""
    response = f"Answer: {example['answer']}\n\nContext:\n{context_text}"
    return {"instruction": example["question"], "response": response}


def format_gpt4(example):
    instruction, response = None, None

    # Check for the 'conversations' key typical of OpenThoughts structure
    if 'items' in example and isinstance(example['items'], list):
        # The user's instruction is the 'value' of the first 'user' turn
        # 1. Extract Instruction (User's turn)
        user_turn = next((c for c in example['items'] if c['from'] == 'human'), None)
        if user_turn:
            instruction = user_turn.get('value')

        # 2. Extract Response (Assistant's turn)
        assistant_turn = next((c for c in example['items'] if c['from'] == 'gpt'), None)
        if assistant_turn:
            response = assistant_turn.get('value')
    
    
    return {"instruction": instruction, "response": response, "source": "sharegpt4"}    
    

# ================================================
# 3️⃣ load data from different Datasets
# ================================================


print("📥 Starting dataset loading process and inspection...")


# Set a consistent seed for reproducible random sampling
SEED = 42 

def load_and_random_select(dataset_name, split_name, percentage, config_name=None):
    """Loads a split, shuffles it, and selects the specified percentage of entries."""
    print(f"\n-> Loading and selecting {percentage*100:.0f}% randomly from {dataset_name} ({split_name})")
    
    # 1. Load the full split
    if config_name:
        ds_full = load_dataset(dataset_name, config_name, split=split_name)
    else:
        ds_full = load_dataset(dataset_name, split=split_name)
    
    # Calculate the number of entries to select
    num_to_select = int(len(ds_full) * percentage)
    
    # 2. Shuffle the dataset and select the required number of entries
    # The 'seed' makes the random selection reproducible
    ds_random = ds_full.shuffle(seed=SEED).select(range(num_to_select))
    
    return ds_random


#Conversational tone / flow  Ultrachat, Dolly 45 %  Teaches helpful, polite phrasing and structure

#dolly = load_dataset("databricks/databricks-dolly-15k", split="train[:80%]")
dolly = load_and_random_select("databricks/databricks-dolly-15k", "train", 0.15)
inspect_random_entry("databricks/databricks-dolly-15k", dolly)

#ultrachat = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft[:10%]")
ultrachat = load_and_random_select("HuggingFaceH4/ultrachat_200k", "train_sft", 0.01)
inspect_random_entry("HuggingFaceH4/ultrachat_200k", ultrachat)

# Structured reasoning (CoT) Open-Thoughts, GSM8K, MATH 35 % Injects step-by-step clarity and logical progression

#cot_thoughts = load_dataset("open-thoughts/open-thoughts-114k", split="train[:21%]")
cot_thoughts = load_and_random_select("open-thoughts/open-thoughts-114k", "train", 0.40)
inspect_random_entry("open-thoughts/open-thoughts-114k", cot_thoughts)

gsm8k = load_dataset("openai/gsm8k", "main", split="train") 
inspect_random_entry("openai/gsm8k", gsm8k)

math_ds = load_dataset("HuggingFaceH4/math", split="train")
inspect_random_entry("HuggingFaceH4/math", math_ds)

# Instruction diversity / broad generality Alpaca, ARC, BoolQ 15 % Keeps model generalist and able to follow any prompt
#alpaca = load_dataset("tatsu-lab/alpaca", split="train[:15%]")
alpaca = load_and_random_select("tatsu-lab/alpaca", "train", 0.05)
inspect_random_entry("tatsu-lab/alpaca", alpaca)

#boolq = load_dataset("google/boolq", split="train[:30%]")
boolq = load_and_random_select("google/boolq", "train", 0.30)
inspect_random_entry("google/boolq", boolq)

arc = load_dataset("allenai/ai2_arc", "ARC-Challenge", split="train")
inspect_random_entry("allenai/ai2_arc", arc)

# Human-preferred style LMSYS winners 5 % “Style anchor” for alignment and empathy
#lmsys_conv_raw = load_dataset("lmsys/chatbot_arena_conversations", split="train[:15%]") 
lmsys_conv_raw = load_and_random_select("lmsys/chatbot_arena_conversations", "train", 0.08)
inspect_random_entry("lmsys/chatbot_arena_conversations", lmsys_conv_raw)


# lmsys/sharegpt4-dataset (subset)
#sharegpt = load_dataset("anon8231489123/ShareGPT_Vicuna_unfiltered", split="train[:3%]")
#sharegpt = load_dataset("LDJnr/ShareGPT-processed", split="train[:5%]")
#sharegpt = load_dataset("lmsys/sharegpt4-dataset", split="train[:5%]")
#inspect_random_entry("lmsys/sharegpt4-dataset", sharegpt)

sharegpt4 = load_dataset(
    "openchat/openchat_sharegpt4_dataset",
    data_files="sharegpt_gpt4.json",
    split="train"
)

inspect_random_entry("openchat/openchat_sharegpt4_dataset/sharegpt_gpt4.json", sharegpt4, force_disp = False)


# ================================================
# 4️⃣ format datasets to instruction:response for SFT
# ================================================


print("\n🧠 Applying OpenThoughts formatter...")
cot_thoughts_formatted = cot_thoughts.map(format_OpenThoughts, remove_columns=cot_thoughts.column_names)
inspect_formatted_sft("cot_thoughts_formatted", cot_thoughts_formatted)

print("\n🧠 Applying gsm8k formatter...")
gsm8k_formatted = gsm8k.map(format_GSM8K, remove_columns=gsm8k.column_names)
inspect_formatted_sft("gsm8k_formatted", gsm8k_formatted, 2, 200)

print("\n🧠 Applying LMSYS winner extraction logic...")
lmsys_conv_formatted = lmsys_conv_raw.map(extract_lmsys_winner, remove_columns=lmsys_conv_raw.column_names)
inspect_formatted_sft("lmsys_conv_formatted", lmsys_conv_formatted)

print("\n🧠 Applying dolly formatter...")
dolly_formatted = dolly.map(format_dolly, remove_columns=dolly.column_names)
inspect_formatted_sft("dolly_formatted", dolly_formatted)

print("\n🧠 Applying ultrachat formatter...")
ultrachat_formatted = ultrachat.map(format_ultrachat, remove_columns=ultrachat.column_names)
inspect_formatted_sft("ultrachat_formatted", ultrachat_formatted)

print("\n🧠 Formatting math formatter...")
math_formatted = math_ds.map(format_math, remove_columns=math_ds.column_names)
inspect_formatted_sft("math_formatted", math_formatted)

print("\n🧠 Formatting alpaca formatter...")
alpaca_formatted = alpaca.map(format_alpaca, remove_columns=alpaca.column_names)
inspect_formatted_sft("alpaca_formatted", alpaca_formatted)

print("\n🧠 Formatting boolq formatter...")
boolq_formatted = boolq.map(format_boolq, remove_columns=boolq.column_names)
inspect_formatted_sft("boolq_formatted", boolq_formatted)

print("\n🧠 Formatting arc formatter...")
arc_formatted = arc.map(format_arc, remove_columns=arc.column_names)
inspect_formatted_sft("arc_formatted", arc_formatted)

print("\n🧠 Formatting arc formatter...")
gpt4_formatted = sharegpt4.map(format_gpt4, remove_columns=sharegpt4.column_names)
inspect_formatted_sft("gpt4_formatted", gpt4_formatted, force_disp = False)

# ================================================
# 5️⃣  Merge datasets together
# ================================================


formatted_datasets = []

formatted_datasets.extend([
    # Training Chain-of-Thought (CoT) and structured output.
    cot_thoughts_formatted,

    # Injecting the human-preferred response style (the "winner's" style) directly from real-world usage data.
    lmsys_conv_formatted,
    
    # Teaches the model to follow multi-step arithmetic and symbolic logic precisely. 
    # Foundation for numerical and multi-step reasoning.
    #gsm8k_formatted,

    # Natural Tone & Diversity. output is conversational, polite, and natural (human-written style)
    dolly_formatted,

    # Provides a massive volume of clean, multi-turn dialogues. Your SFT extraction
    # flattens these into single turns, but the model still learns the density of
    # conversation—the expectation of clear, concise, and structured responses in a
    # chat environment.
    ultrachat_formatted,
    
    #math_formatted,

    # instruction-following dataset 
    alpaca_formatted,

    # designed for Question Answering (QA) tasks
    # Implicit Reasoning: Answering BoolQ often requires more complex reasoning and
    # inference than simply locating a specific keyword or phrase. 
    boolq_formatted,

    # AI2 ARC (AI2 Reasoning Challenge) is a benchmark dataset created by the Allen
    # Institute for AI to evaluate AI systems on grade-school level science reasoning.
    arc_formatted,

    # High-quality instruction–response pairs
    # GPT-4-level responses (high reasoning, coherence, and helpfulness)
    gpt4_formatted,
])


mixed = concatenate_datasets(formatted_datasets).shuffle(seed=42)

# ================================================
# 6️⃣  merged datasets filter
# ================================================

print("🗑️ Applying quality filter...")

def filter_null_or_empty(example):
    inst = example.get("instruction")
    resp = example.get("response")
    
    if not isinstance(inst, str) or not isinstance(resp, str):
        return False
    if len(inst.strip()) < 5 or len(resp.strip()) < 2:
        return False
        
    return True

mixed = mixed.filter(filter_null_or_empty)


MAX_SAMPLES = 38000 

# Shuffle the dataset first (use a seed for reproducibility)
shuffled_dataset = mixed.shuffle(seed=42)

# Then, select the first N samples from the now-shuffled dataset
dataset_random = shuffled_dataset.select(range(min(MAX_SAMPLES, len(shuffled_dataset))))

# take all of gsmk8 math
# combined_dataset = concatenate_datasets([dataset_random, gpt4_formatted, gsm8k_formatted, math_formatted ])
combined_dataset = concatenate_datasets([dataset_random, gsm8k_formatted, math_formatted ])

from transformers import AutoTokenizer
import pandas as pd

MAX_SEQ_LEN = 4096

# Define your custom system prompt
CUSTOM_SYSTEM_PROMPT = """\
You are a highly professional, concise technical expert across modern computing domains — 
including software architecture, cloud infrastructure, data systems, machine learning, and applied AI.

Your task is to:
- Answer the user’s question using the provided CONTEXT as your primary source.
- If the CONTEXT does not contain enough information, use your own knowledge,
  but clearly distinguish between context-based and general reasoning.

Your responses must be:
- Structured — use clear formatting and logical reasoning.
- Contextual — rely only on the information available.
- Concise — eliminate filler words while preserving precision.
- Aligned with industry best practices — modern, reproducible, and standards-based.
"""


from transformers import AutoTokenizer

# Just load the tokenizer — no GPU or model weights needed
tokenizer = AutoTokenizer.from_pretrained(
    #"unsloth/gpt-oss-20b-unsloth-bnb-4bit",
    "unsloth/gpt-oss-20b",
    
     # Use Fast tokenizer
    use_fast=True,     
    
    # Required for Unsloth custom templates
    trust_remote_code=True,  
)


# Add customized prompt and apply chat template, then use tokenizer compute the length to token
# Only accept length <= 4096 (max_seq_length)
def is_length_within_limit(example):
    # Create a Harmony-format conversation
    messages = [
        # {"role": "system", "content": CUSTOM_SYSTEM_PROMPT},
        {"role": "user", "content": example.get("instruction", "")},
        {"role": "assistant", "content": example.get("response", "")},
    ]
    wrapped = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=False,
        tokenize=True,
    )
    #print(wrapped)
    token_count = len(wrapped["input_ids"]) if isinstance(wrapped, dict) else len(wrapped)
    return token_count <= MAX_SEQ_LEN



# Use the .filter() method which returns a new Dataset object
filtered_dataset = combined_dataset.filter(
    is_length_within_limit,
    num_proc=4 # Use multiple processes to speed up tokenization/filtering
)

# .shuffle() on the Dataset object
final_dataset_random = filtered_dataset.shuffle(seed=42)

print("\n✅ Final dataset successfully filtered and shuffled.")


# Prints parts of the datasets with chat template
def inspect_formatted_sft(ds_name, ds_object, n = 2, force_disp = False):

    if not force_disp:
        if not INSPECT_ON:
            return 
    
    for i, entry in enumerate(ds_object):
        if i > n:
            break
        print(f"--- Entry {i+1} ---")

            # Create a Harmony-format conversation
        messages = [
            {"role": "system", "content": CUSTOM_SYSTEM_PROMPT},
            {"role": "user", "content": entry.get("instruction", "")},
            {"role": "assistant", "content": entry.get("response", "")},
        ]
        wrapped = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=False,
            tokenize=False,
        )
        
        print(f"Message with chat template: {wrapped}...") 

        print("-" * 50)
        print("\n")


inspect_formatted_sft("final_dataset_random", final_dataset_random, force_disp = True)

# ================================================
# 7️⃣  Analysis datasets 1. percentage of each source 2. max length(token) of each source
# ================================================


# Analysis: Getting the Source Statistics
print("\n📊 Analyzing Sources in Final Dataset...")

# Extract the 'source' column into a list
sources_list = final_dataset_random['source']

# Get the total size of the final dataset
final_dataset_size = len(final_dataset_random)

# Convert the list to a Pandas Series and count the unique values (This gives the 'Count' column)
source_counts = pd.Series(sources_list).value_counts()

# Calculate the total number of unique sources
total_unique_sources = len(source_counts)

# Create a DataFrame for summary and add the Percentage column 
# Convert the Series to a DataFrame
summary_df = source_counts.rename('Count').to_frame()

# Calculate the percentage
summary_df['Percentage'] = (summary_df['Count'] / final_dataset_size) * 100

# Format the Percentage column for clean display
summary_df['Percentage'] = summary_df['Percentage'].map('{:.2f}%'.format)

# Printing the Summary
print("-" * 50)
print(f"Final Dataset Size: {final_dataset_size} entries")
print(f"Total Unique Sources Found: {total_unique_sources}")
print("-" * 50)
print("Count and Percentage of Data Entries per Source:")
print(summary_df.to_string()) # Print the DataFrame with both Count and Percentage
print("-" * 50)


# Save as JSONL for Unsloth
df = pd.DataFrame(final_dataset_random)
df.to_json("train_sft_final.json", orient="records", lines=True, force_ascii=False)

print(f"\n🎉 SUCCESS! Final SFT dataset created: train_sft_final.json")
print(f"Total samples after filtering: {len(final_dataset_random)}")

📥 Starting dataset loading process and inspection...

-> Loading and selecting 15% randomly from databricks/databricks-dolly-15k (train)

-> Loading and selecting 1% randomly from HuggingFaceH4/ultrachat_200k (train_sft)

-> Loading and selecting 40% randomly from open-thoughts/open-thoughts-114k (train)

-> Loading and selecting 5% randomly from tatsu-lab/alpaca (train)

-> Loading and selecting 30% randomly from google/boolq (train)

-> Loading and selecting 8% randomly from lmsys/chatbot_arena_conversations (train)

🧠 Applying OpenThoughts formatter...

🧠 Applying gsm8k formatter...

🧠 Applying LMSYS winner extraction logic...

🧠 Applying dolly formatter...

🧠 Applying ultrachat formatter...

🧠 Formatting math formatter...

🧠 Formatting alpaca formatter...

🧠 Formatting boolq formatter...

🧠 Formatting arc formatter...

🧠 Formatting arc formatter...
🗑️ Applying quality filter...

✅ Final dataset successfully filtered and shuffled.
--- Entry 1 ---
Message with chat template: <|start|

In [19]:


# Alternatively, a cleaner map that keeps only source and tokens
def prepare_data_for_analysis(example, tokenizer):
    combined_text = example['instruction'] + " " + example['response']
    tokenized = tokenizer(combined_text, truncation=False, return_length=True)
    return {
        'source': example['source'], # Assuming you have a 'source' column
        'total_tokens': tokenized['length'][0]
    }

analysis_dataset = final_dataset_random.map(
    lambda x: prepare_data_for_analysis(x, tokenizer),
    batched=False, # Process one example at a time
    num_proc=4,
    remove_columns=final_dataset_random.column_names # Keep only the two new columns
)

# Convert the analysis dataset to a Pandas DataFrame
analysis_df = analysis_dataset.to_pandas()

# Group by the 'source' column and find the maximum 'total_tokens'
max_lengths_by_source = analysis_df.groupby('source')['total_tokens'].max().sort_values(ascending=False)

## Output the result
print("Maximum Token Length (Instruction + Response) per Source:")
print("-------------------------------------------------------")
print(max_lengths_by_source)


# Convert to Pandas for aggregation
analysis_df = analysis_dataset.to_pandas()

# Filter for entries where token length is greater than the threshold
large_entries_df = analysis_df[analysis_df['total_tokens'] > MAX_SEQ_LEN]

#  Group by 'source' and count the number of entries
count_by_source = large_entries_df.groupby('source').size().sort_values(ascending=False)

# Print the final result
print(f"Count of Entries with Token Length > {MAX_SEQ_LEN} per Source:")
print(count_by_source)






Maximum Token Length (Instruction + Response) per Source:
-------------------------------------------------------
source
OpenThoughts    4025
sharegpt4       3728
dolly           3337
math            2445
ultrachat       2366
lmsys_conv      1256
boolq            580
alpaca           444
GSM8K            409
arc              235
Name: total_tokens, dtype: int64
Count of Entries with Token Length > 4096 per Source:
Series([], dtype: int64)
